In [1]:
import os
from itertools import chain
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, LSTM, TimeDistributed, Dropout, Dense, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator, array_to_img
import numpy as np
from PIL import Image
from tqdm import tqdm
from tqdm.keras import TqdmCallback
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pandas as pd
from typing import List

from data import Data
from metrics import SlicewiseAccuracy
from data_generator import PhaseDataGenerator, SliceDataGenerator
from results import *

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Init Backbone

In [3]:
keras_app = tf.keras.applications.mobilenet
keras_model = tf.keras.applications.mobilenet.MobileNet
backbone = keras_model(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
backbone.trainable = False

## Initializer Data Pipeline

In [4]:
data = Data()

In [5]:
dgen_train= ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1,
    fill_mode="nearest",
    preprocessing_function=keras_app.preprocess_input)
dgen_val = ImageDataGenerator(preprocessing_function=keras_app.preprocess_input)
pgen_train = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2,
                               shuffle=True, image_data_generator=dgen_train,
                               split_index=0)
pgen_val = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2,
                              shuffle=False, image_data_generator=dgen_val,
                              split_index=1)
sgen_train = SliceDataGenerator(data, "KAG", target_size=(224, 224), batch_size=32,
                                shuffle=True, image_data_generator=dgen_train,
                                split_index=0)
sgen_val = SliceDataGenerator(data, "KAG", target_size=(224, 224), batch_size=32,
                              shuffle=False, image_data_generator=dgen_val,
                              split_index=1)

In [6]:
assert(len(set(pgen_train.sample_keys).intersection(pgen_val.sample_keys)) == 0)

In [7]:
assert(len(set(sgen_train.slices).intersection(sgen_val.slices)) == 0)

In [8]:
assert(np.isclose(sgen_train[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(sgen_train[0][0][0].max(), 1, rtol=1.0e-4))
assert(np.isclose(pgen_train[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(pgen_train[0][0][0].max(), 1, rtol=1.0e-4))

## End-To-End Models

In [9]:
cnn_model = Sequential()
cnn_model.add(backbone)
cnn_model.add(Dense(256, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5, activation="softmax"))

cnn_model.layers[0].trainable = False
cnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [10]:
cnn_history = cnn_model.fit(sgen_train, validation_data=sgen_val, epochs=1)

449/449 [==============================] - 347s 773ms/step - loss: 0.9051 - accuracy: 0.6245 - val_loss: 0.6362 - val_accuracy: 0.7392


In [11]:
rnn_model = Sequential()
rnn_model.add(TimeDistributed(backbone))
rnn_model.add(LSTM(256, input_shape=(25, 2048), return_sequences=True))
rnn_model.add(Dropout(0.5))
#rnn_model.add(TimeDistributed(Dense(256, activation="relu")))
#rnn_model.add(TimeDistributed(Dropout(0.5)))
rnn_model.add(TimeDistributed(Dense(5, activation="softmax")))

rnn_model.layers[0].trainable = False
rnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=[SlicewiseAccuracy()])

In [12]:
rnn_history = rnn_model.fit(pgen_train, validation_data=pgen_val, epochs=1)

682/682 [==============================] - 792s 1s/step - loss: 0.2750 - accuracy: 0.7268 - val_loss: 0.2186 - val_accuracy: 0.7819


## Evaluate

In [13]:
print(cnn_model.evaluate(sgen_val))
print(rnn_model.evaluate(pgen_val))
cnn_preds = cnn_model.predict(sgen_val)
rnn_preds = rnn_model.predict(pgen_val)

194/194 [==============================] - 105s 539ms/step - loss: 0.6362 - accuracy: 0.7392
[0.6361936330795288, 0.739158570766449]
292/292 [==============================] - 237s 812ms/step - loss: 0.2186 - accuracy: 0.7819
[0.2186177372932434, 0.7818770408630371]


In [14]:
cnn_preds.shape, rnn_preds.shape

((6180, 5), (584, 25, 5))

In [15]:
slice_index = pgen_val.get_slice_index()
rnn_predictions = compile_predictions_from_phase_output(data, rnn_preds, slice_index)

In [16]:
slice_list = sgen_val.get_slice_list()
cnn_predictions = compile_predictions_from_slice_output(data, cnn_preds, slice_list)

### Validate Evaluation

In [17]:
def get_accuracy(preds, labels):
    total_slices = 0
    total_phases = 0
    correct_slices = 0
    correct_phases = 0

    for phase, slices in enumerate(preds):
        total_phases += 1
        correct_phase = True
        for slice, pred in enumerate(slices):
            label = labels[phase][slice]
            if label.sum() == 0:
                pass
            else:
                total_slices += 1
                if label.argmax() == pred.argmax():
                    correct_slices += 1
                else:
                    correct_phase = False
        if correct_phase:
            correct_phases += 1

    slice_accuracy = correct_slices / total_slices
    phase_accuracy = correct_phases / total_phases

    return slice_accuracy, phase_accuracy, total_slices

In [18]:
# Validate results
y_rnn = np.ndarray((0, 25, 5))
for _, labels in tqdm(pgen_val):
    y_rnn = np.concatenate([y_rnn, labels])
get_accuracy(rnn_preds, y_rnn)

100%|██████████| 292/292 [00:25<00:00, 11.49it/s]


(0.7818770226537217, 0.0702054794520548, 6180)

In [19]:
cnn_result = evaluate_predictions(cnn_predictions)
cnn_result

{'accuracy': 0.7391585760517799, 'phase_accuracy': 0.046232876712328765}

In [20]:
rnn_result = evaluate_predictions(rnn_predictions)
rnn_result

{'accuracy': 0.7818770226537217, 'phase_accuracy': 0.0702054794520548}

## Save Results

In [21]:
result_key = "initial_rnn"
os.makedirs(get_result_dir(result_key), exist_ok=True)

In [22]:
for key, value in rnn_predictions.items():
    scores = value["scores"]
    for k, v in scores.items():
        scores[k] = str(v)

In [23]:
with open(get_history_path(result_key), "w") as f:
    json.dump(rnn_history.history, f)
with open(get_predictions_path(result_key), "w") as f:
    json.dump(rnn_predictions, f)
with open(get_result_path(result_key), "w") as f:
    json.dump(rnn_result, f)
rnn_model.save_weights(get_model_weights_path(result_key))

In [24]:
result_key = "initial_cnn"
os.makedirs(get_result_dir(result_key), exist_ok=True)

In [25]:
with open(get_history_path(result_key), "w") as f:
    json.dump(cnn_history.history, f)
with open(get_predictions_path(result_key), "w") as f:
    json.dump(cnn_predictions, f)
with open(get_result_path(result_key), "w") as f:
    json.dump(cnn_result, f)
cnn_model.save_weights(get_model_weights_path(result_key))